### Predicting the Dynamics after Object Interactions

We developed an environment containing a number of balls bouncing around in a frictionless 2D plane with elastic collisions between objects (i.e., total conservation of energy). This environment was represented at each time-step using a single SSP (see Figure 6). We found that to get the scales to match, the SSP coordinates need to be scaled by $\sqrt{2}$.

We then trained models to predict the velocity of each object after any interaction (either between an object and the wall, or between two objects that have collided), given the current state of the system (represented using one SSP encoding all of the positions, and another SSP encoding all of the velocities).

We found that a single ReLU perceptron is able to correctly predict the angles of each object’s velocity following any object interaction. However, the magnitudes of the velocities were difficult to accurately predict. Future work should consider scaling up these capabilities and systematically evaluating performance. 

In [ ]:
%matplotlib inline

In [ ]:
import string

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML

import nengo_spa as spa
from ssp.cleanup import Cleanup
from ssp.collisions import generate_collision_data, Simulation
from ssp.maps import Spatial2D
from ssp.models import MLP
from ssp.plots import heatmap_animation, create_gif

In [ ]:
# TODO: refactor common code with benchmarking into ssp/collisions.py

grid_size = 5  # in units of circle's diameter
n_particles = 3
radii = np.ones(n_particles) / grid_size / 2
dt = 0.2
frames = 500
interval = 40

In [ ]:
sim = Simulation(n_particles, radius=radii, rng=np.random.RandomState(seed=0))
ani = sim.do_animation(dt=dt, frames=frames, interval=interval)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani, fname="ideal.gif")))

In [ ]:
# points in the simulation are within [0, 1]^2 and then visualized
# on [-0.5, 0.5]^2 with a scale of sqrt(2)*grid_size such
# that the diameter of each ball is roughly the same scale

dim = 1024
ssp_radius = np.sqrt(2)  # open problem: deriving this
ssp_scale = ssp_radius * grid_size

ssp_map = Spatial2D(dim=dim, scale=ssp_scale, rng=np.random.RandomState(seed=0))
ssp_map.build_grid(x_len=0.5, y_len=0.5, x_spaces=101, y_spaces=101, centered=True)

names = string.ascii_uppercase[:n_particles]
assert len(names) == n_particles

position_offset = -0.5

In [ ]:
sim = Simulation(n_particles, radius=radii, rng=np.random.RandomState(seed=0))

heatmaps = []
for step in range(frames):    
    ssp = ssp_map.encode_points(
        sim.x + position_offset, sim.y + position_offset, names,
    )
    sim.advance(dt)
    heatmaps.append(ssp_map.compute_heatmap(ssp, names))

ani = heatmap_animation([heatmaps], figsize=(4, 4), interval=interval)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani, fname="ssp.gif")))

In [ ]:
sim = Simulation(n_particles, radius=radii, rng=np.random.RandomState(seed=0))

X, Y, (low, high) = generate_collision_data(
    sim=sim,
    dt=dt,
    ssp_map=ssp_map,
    names=names,
    position_offset=position_offset,
    n_collisions=10000,
)

print(low, high)

In [ ]:
split = int(0.8 * len(X))
train_X, test_X = X[:split], X[split:]
train_Y, test_Y = Y[:split], Y[split:]

In [ ]:
model = MLP(2*dim, [10000], dim)
model.train(train_X, train_Y, n_steps=1000)

In [ ]:
test_Yhat = model(test_X)
test_cost = model.cost(test_Yhat, test_Y)

In [ ]:
plt.figure()
plt.plot(model.costs, label="Train")
plt.hlines([test_cost], 0, len(model.costs) - 1, label="Test")
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
v_cleanup = Cleanup(model=dim, vocab=ssp_map.voc)
v_cleanup.train(objs=names, low=low, high=high, n_steps=500)

In [ ]:
p_cleanup = Cleanup(model=dim, vocab=ssp_map.voc)
# [low, high] combines both [-0.5, 0.5]^2 grid and ssp map scale
p_cleanup.train(objs=names, low=-ssp_scale/2, high=ssp_scale/2, n_steps=500)

In [ ]:
sim = Simulation(n_particles, radius=radii, rng=np.random.RandomState(seed=42))

last_state = None
Vhats = None
Phats = None

heatmaps = []
for step in range(frames):
    # TODO: refactor this into ssp/collisions.py
    P = ssp_map.encode_points(
        sim.x + position_offset, sim.y + position_offset, names,
    )

    if Vhats is None:
        # names are not needed here since these will be bound elementwise with P
        Vhats = [
            ssp_map.encode_point(dt * vx, dt * vy, name=None)
            for vx, vy in zip(sim.vx, sim.vy)
        ]

    if Phats is None:
        Phats = [
            ssp_map.encode_point(
                x + position_offset, y + position_offset, name=name)
            for x, y, name in zip(sim.x, sim.y, names)
        ]
    else:
        for i in range(n_particles):
            Phats[i] = Phats[i] * Vhats[i]
            Phats[i] = ssp_map.voc[names[i]] * p_cleanup(Phats[i] * ~ssp_map.voc[names[i]])
            Phats[i].name = ""

    heatmap_ideal = ssp_map.compute_heatmap(P, names)
    heatmap_pred = ssp_map.compute_heatmap(np.sum(Phats), names)
    heatmaps.append([heatmap_ideal, heatmap_pred])

    this_state = np.vstack([sim.vx, sim.vy])
    if last_state is not None and not np.allclose(this_state, last_state):
        V_prev = ssp_map.encode_points(
            dt * last_state[0], dt * last_state[1], names,
        )
        ssp_pred = spa.SemanticPointer(model(np.hstack([P.v, V_prev.v])))
        
        Phats = None
        Vhats = [
            v_cleanup(ssp_pred * ~ssp_map.voc[name])
            for name in names
        ]
    last_state = this_state

    sim.advance(dt)

ani = heatmap_animation(list(zip(*heatmaps)), figsize=(8, 4), interval=interval)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani, fname="ssppred.gif")))